# ST7 Planification quotidienne d’une équipe mobile

In [1]:
# Modules de base
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Module relatif à Gurobi
from gurobipy import *

## Formulation du problème

### Notation
- $ V $ : Nombre de tâches + 1 (V pour vertex)
- $ T $ : Nombre de techniciens
- $ i, j \in \{0, ..., V - 1\} $ : les indices des tâches
- $ k \in \{0, ..., T - 1 \} $ : les indices des techniciens
- $ (opening_i)_i $ : l’ouverture des sites
- $ (closing_i)_i $ : la fermeture des sites
- $ (start_k)_k $ : le début de travail des employés
- $ (end_k)_k $ : la fin de travail de chaque employé
- $ (duration_i)_i $ : la durée de chaque tâche
- $ (distance_{i, j}) $ : la distance entre les sites
- $ speed $ : la vitesse de déplacement des techniciens
- $ M $ : majorant pour notre problème



### Variables de décisions
- $ \tilde{T} $ : Le nombre de techniciens actifs (avec une tâche ou plus)
- $ (x_{i, j})_{i, j} $, $ x_{i, j} = 1 $ ssi. (i, j) est un arc de i vers j
- $ (y_{k, i})_{k, i} $, $ y_{k, i} = 1 $ ssi. le technicien k effectue la tâche i
- $ (b_i)_i $, $B_i$ est le début de chaque tâche en minute (b pour beginning)


### Objectif d’optimisation
$$ min \sum_{i, j} x_{i, j} \cdot distance_{i, j} $$


### Contraintes
- (C1) : Il n’y a qu’une seule composante connexe dans le graphe des parcours
$$ \sum_{i, j} x_{i, j} = \tilde{T} + (V - 1) $$
- (C2) Il y a autant d’arcs sortant/entrant au dépôt que de techniciens actifs
$$ \tilde{T} = \sum_i x_{0, i} = \sum_i x_{i, 0}$$
- (C3) Le site de chaque tâche est visité par exactement un technicien
$$ \forall i > 0, \sum_j x_{i, j} = \sum_j x_{j, i} = 1 $$
- (C4) Deux tâches qui se suivent doivent être faites par le même technicien
$$ \forall i, j > 0, x_{i, j} \implies \forall k, y_{k, i} = y_{k, j}$$
- (C5) Lorsqu’un travail est effectué sur une tâche, la tâche doit être disponible.
$$ \forall i > 0, (b_i \ge opening_i) \land (b_i + duration_i \le closing_i) $$
- (C6) La fenêtre de temps entre deux travaux doit être suffisante pour le trajet
$$ \forall i, j > 0, x_{i, j} = 1 \implies b_i + duration_i + \frac{distance_{i, j}}{speed} \le b_j \\
\Longleftrightarrow \forall i, j > 0, b_i + duration_i + \frac{distance_{i, j}}{speed} \le b_j + M \cdot (1-x_{i, j}) $$
- (C7) Un technicien doit avoir suffisamment de temps pour aller à son premier site
$$ \forall k > 0, \forall i > 0, y_{k, i} \land x_{0, i} \implies start_k + \frac{distance_{0, i}}{speed} \le b_i \\
\Longleftrightarrow \forall k > 0, \forall i > 0, start_k + \frac{distance_{0, i}}{speed} \le b_i + (2 - y_{k, i} - x_{0, i}) \cdot M $$
- (C8) Un technicien doit avoir suffisamment de temps pour rentrer au dépot après sa dernière tâche
$$ \forall k > 0, \forall i > 0, y_{k, i} \land x_{i, 0} \implies b_i + duration_i + \frac{distance_{i, 0}}{speed} \le end_k \\
\Longleftrightarrow \forall k > 0, \forall i > 0, b_i + duration_i + \frac{distance_{i, 0}}{speed} \le end_k + (2 - y_{k, i} - x_{i, 0}) \cdot M $$
- (C9) Un technicien n’effectue que des tâches qu’il est capable d’effectuer
$$ \forall k > 0, \forall i > 0, levelTech_k \ge levelTask_i - M \cdot (1 - y_{k, i}) $$
- (C10) Une tâche est réalisée par un seul employé
$$ \forall i > 0, \sum_k y_{k, i} = 1 $$



## Variables de décisions
$ X \in R_{V + 1 \times V + 1}$

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from math import radians, cos, sin, asin, sqrt
from utils import parse_time_minute, parse_time


class Employee:
    list = []  # initialized to empty list
    count = 0
    speed = 50 * 1000 / 60  # unit: meter/minute

    def __init__(self, name: str, latitude: float, longitude: float, skill: str, level: int, start_time, end_time):
        self.name = name
        self.latitude = latitude
        self.longitude = longitude
        self.skill = skill
        self.level = level
        self.start_time_str = parse_time(start_time)  # parse time into datetime object for printing
        self.end_time_str = parse_time(end_time)
        self.start_time = parse_time_minute(start_time)  # parse time into minutes
        self.end_time = parse_time_minute(end_time)
        Employee.count += 1
        Employee.list.append(self)

    @classmethod
    def load_excel(cls, path):
        df_employees = pd.read_excel(path, sheet_name="Employees")
        df_employees.set_index("EmployeeName")

        for index, row in df_employees.iterrows():
            Employee(row["EmployeeName"],
                     row["Latitude"],
                     row["Longitude"],
                     row["Skill"],
                     row["Level"],
                     row["WorkingStartTime"],
                     row["WorkingEndTime"])

    def __hash__(self):
        return hash(self.name)

    def __eq__(self, other):
        return self.name == other.name

    def __str__(self):
        return self.name

    def __repr__(self):
        return f"Employee(name={self.name}, " \
               f"position=[{self.longitude}, {self.latitude}], " \
               f"skill_requirement=level {self.level} {self.skill}," \
               f"available=[{self.start_time_str.strftime('%I:%M%p')}, {self.end_time_str.strftime('%I:%M%p')}] )"


class Task:
    list = []
    count = 0
    distance: np.array = None
    __is_initialized = False

    def __init__(self, task_id, latitude, longitude, duration, skill, level, opening_time, closing_time):
        if Task.__is_initialized:
            raise Exception("Cannot instantiate new task after initializing the distance matrix")
        self.id = task_id
        self.latitude = latitude
        self.longitude = longitude
        self.duration = duration
        self.skill = skill
        self.level = level
        self.opening_time_str = parse_time(opening_time)
        self.closing_time_str = parse_time(closing_time)
        self.opening_time = parse_time_minute(opening_time)
        self.closing_time = parse_time_minute(closing_time)

        Task.list.append(self)
        Task.count += 1

    @classmethod
    def load_excel(cls, path, initialize_distance=False, load_depot=False):

        # create a dummy task at position 0 for depot
        if load_depot:
            df_employees = pd.read_excel(path, sheet_name="Employees")
            depot_longitude = df_employees.iloc[0]["Longitude"]
            depot_latitude = df_employees.iloc[0]["Latitude"]
            Task("T0", depot_latitude, depot_longitude, 0, None, 0, None, None)

        df = pd.read_excel(path, sheet_name="Tasks")
        df.set_index("TaskId")

        for index, row in df.iterrows():
            # parse the start time and end time into datetime object
            opening_time = datetime.strptime(row["OpeningTime"], '%I:%M%p')
            closing_time = datetime.strptime(row["ClosingTime"], '%I:%M%p')

            Task(row["TaskId"],
                 row["Latitude"],
                 row["Longitude"],
                 row["TaskDuration"],
                 row["Skill"],
                 row["Level"],
                 opening_time,
                 closing_time)

        if initialize_distance:
            cls.initialize_distance()

    @staticmethod
    def calculate_distance(task1, task2):
        lon1, lat1 = radians(task1.longitude), radians(task1.latitude)
        lon2, lat2 = radians(task2.longitude), radians(task2.latitude)

        # Haversine formula
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
        c = 2 * asin(sqrt(a))
        r = 6371 * 1000  # radius of earth
        return c * r

    @classmethod
    def initialize_distance(cls):
        if cls.__is_initialized:
            raise Exception("Distance has already been initialized")
        cls.__is_initialized = True
        cls.distance = np.zeros((cls.count, cls.count), dtype=np.float64)

        for i in range(cls.count):
            for j in range(i):
                task_i, task_j = cls.list[i], cls.list[j]
                cls.distance[i, j] = cls.distance[j, i] = cls.calculate_distance(task_i, task_j)

    def __hash__(self):
        return hash(self.id)

    def __eq__(self, other):
        return self.id == other.id

    def __repr__(self):
        return f"Task(id={self.id}, " \
               f"position=[{self.longitude}, {self.latitude}], " \
               f"duration={self.duration}, " \
               f"skill_requirement=level {self.level} {self.skill}," \
               f"opening_time=[{self.opening_time_str.strftime('%I:%M%p')} to {self.closing_time_str.strftime('%I:%M%p')}]"


In [3]:
# module imports
# from models_v1 import Employee, Task
from file_paths import path_finland
from gurobipy import Model, GRB

In [4]:
# reading dataframe into python objects
Employee.load_excel(path_finland)
Task.load_excel(path_finland, initialize_distance=True, load_depot=True)

In [5]:
V = len(Employee.list)
T = len(Task.list)

In [6]:
# test on Task object
Task.list[1]

Task(id=T1, position=[24.850606, 61.178837], duration=40, skill_requirement=level 1 Plumbing,opening_time=[08:00AM to 06:00PM]

In [7]:
V = Task.count
T = Employee.count
(V, T)

(26, 4)

In [8]:
Task.distance

array([[    0.        , 48375.80567912, 35201.43297555, 36519.64027781,
        36519.64027781, 46480.09943848, 41099.14286807, 34079.67007868,
        43579.40710533, 45969.10518946, 33802.15164739, 30159.61306314,
        30159.61306314, 30104.64296159, 28882.73717912, 34580.30227993,
        28696.90658324, 28696.90658324,  1069.30151348, 45291.68637136,
        44571.40787799, 42458.8431502 , 21561.59842308, 21561.59842308,
        21561.59842308,  2022.83976311],
       [48375.80567912,     0.        , 19831.54439314, 15925.08267198,
        15925.08267198,  3124.49393092,  9266.67972921, 27039.20546763,
        12154.80032663,  7693.11944183, 79034.58469799, 27162.27787446,
        27162.27787446, 20096.97052889, 27377.24959714, 14210.30215964,
        27367.83814303, 27367.83814303, 47450.59580049, 19708.74021684,
        25269.06781593, 19736.41333726, 31516.12388168, 31516.12388168,
        31516.12388168, 46722.90102255],
       [35201.43297555, 19831.54439314,     0.        

In [9]:
m = Model("Model-V1")

X = m.addMVar(shape=(V, V), name="adjacence-matrix", vtype=GRB.BINARY)
Y = m.addMVar(shape=(T, V), vtype=GRB.BINARY)
B = m.addMVar(shape=(V,), vtype=GRB.INTEGER, lb=0, ub=1440)

m.addConstr(X.sum() == T + V - 1, name="C1")
m.addConstr(X[0, :].sum() == T, name="C2")
m.addConstrs((X[i, :].sum() == X[:, i].sum() for i in range(0,V)), name="C3")

#C4
for i in range(V):
    for j in range(V):
        m.addConstrs((X[i, j] == 1) >> (Y[k, i] == Y[k, j]) for k in range(T))

opening_time = np.array([Task.list[i].opening_time for i in range(V)])
closing_time = np.array([Task.list[i].closing_time for i in range(V)])
duration = np.array([Task.list[i].duration for i in range(V)])

m.addConstr(B[1:] >= opening_time[1:], name="C5a")
m.addConstr(B[1:] + duration[1:] <= closing_time[1:], name="C5b")

#c6
for i in range(1,V):
    for j in range(1,V):
        m.addConstr((X[i, j] == 1) >> (B[i] + duration[i] + Task.distance[i,j] * Employee.speed <= B[j]))

#c7
for k in range(T):
    for i in range(1, V):
        m.addConstr(Employee.list[k].level >= Task.list[i].level)


last_task = m.addMVar((T, V), vtype=GRB.BINARY)
for i in range()

first_task = m.addMVar((T, V), vtype=GRB.BINARY)
#c8
for i in range(1, V):
    for k in range(T):
        m.addConstr((X[i, 0] + Y[k, i] == 2) >> (Employee.list[k] >= B[i] + Task.distance[i, 0] * Employee.speed + duration[i]))

#c9

#X * Task.distance

SyntaxError: invalid syntax (Temp/ipykernel_10180/3714848579.py, line 35)

In [10]:
m = Model("DB")
X = {(i, j) : m.addVar(vtype = GRB.BINARY, name = f'x{i}_{j}') for i in range(0,V) for j in range(0,V)}
Y = {(k, i) : m.addVar(vtype = GRB.BINARY, name = f'y{k}_{i}') for k in range(0,T) for i in range(0,V)}
B = {i : m.addVar(vtype = GRB.CONTINUOUS, name = f'b{i}') for i in range(0,V)}
T_actif = m.addVar(vtype = GRB.INTEGER, name = 'T_actif')

Set parameter Username
Academic license - for non-commercial use only - expires 2023-01-27


In [11]:
#C1
C1 = m.addConstr(quicksum([X[(i,j)] for i in range(0,V) for j in range(0,V)]) == T_actif + V - 1, name ="C1")

In [12]:
#C2
C2 = m.addConstr(quicksum([X[(i,0)] for i in range(V)]) == T_actif, name="C2")
C2b = m.addConstr(quicksum([X[(0,i)] for i in range(V)]) == T_actif, name="C2b")

In [13]:
#C3
C3_entrée = dict()
C3_sortie = dict()
for i in range(0,V):
    C3_entrée[i] = m.addConstr(quicksum([X[(i,j)] for j in range(0,V)]) == 1, name = f'c3e{i}')
    C3_sortie[i] = m.addConstr(quicksum([X[(j,i)] for j in range(0,V)]) == 1, name = f'c3s{i}')

In [14]:
#C4
C4 = {(i,j,k) : m.addConstr(Y[(k,i)] <= Y[(k,j)] + M*(1-X[(i,j)]), name="C4") for i in range(1,V) for j in range(1,V) for k in range(T)}
C4b = {(i,j,k) : m.addConstr(Y[(k,i)] >= Y[(k,j)] + M*(1-X[(i,j)]), name="C4b") for i in range(1,V) for j in range(1,V) for k in range(T)}


NameError: name 'M' is not defined

In [15]:
#C5
C5 = dict()
for i in range(0,V):
    

IndentationError: expected an indented block (Temp/ipykernel_10180/3781211766.py, line 4)

In [16]:
#C6
C6 = {(i,j) : m.addConstr(B[i] + duration[i] + distance[i][j]/speed <= B[j] - M*(1-X[(i,j)]), name="C6") for i in range(1,V) for j in range(1,V)}

NameError: name 'duration' is not defined

In [ ]:
#C8
C8 = {(k,i) : m.addConstr(B[i] + duration[i] + distance[0][i]/speed <= end[k] + M*(2-Y[(k,i)]-X[(i,0)]), name="C8") for k in range(T) for i in range(1,V)}


In [ ]:
#C10
C10 = {i : m.addConstr(quicksum([Y[(k,i)] for k in range(T)]) ,name="C10") for i in range(1,V)}